In [40]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pickle
from scipy.cluster.vq import *
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn import metrics

train_clf_sift= joblib.load("svc.pkl")

In [2]:
def read(txt, X, Y):
    fp = open(txt, 'r')
    for line in iter(fp):
        X_tmp,Y_tmp = [i for i in line.split()]
        X.append(X_tmp)
        Y.append(Y_tmp)
    fp.close
    return

def sift(des_list, X, Y):
    a = 0
    sift = cv2.xfeatures2d.SIFT_create()
    print("Calculating all the descriptors and keypoints...")
    for i in X:
        a += 1
        print("Calculating for image "+str(a)+"/"+str(len(X)), end="\r")
        im  = cv2.imread(i)
        im = cv2.resize(im, (64, 64))
        gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY) 
        kp, des = sift.detectAndCompute(gray,None)
        des_list.append((i, kp, des))
    a2=0
    
    for j, j1, j2 in des_list[1:]:
        if j2 is None:
            a2 = a2+1
            del des_list[a2]
            del X[a2]
            del Y[a2]
            a2 = a2-1
        else:
            a2 = a2+1
            print("Calculating for image "+str(a2)+"/"+str(len(des_list)), end="\r")
    return

def knn_sift(des_list):
    a1 = 0
    print("Stacking all the descriptors in a numpy array...")
    descriptors = des_list[0][2]
    for image_path, keypoints, descriptor in des_list[1:]:
        a1=a1+1
        print("Calculating for image ["+ str(a1)+"/"+str(len(des_list)) +"]", end="\r")
        descriptors = np.vstack((descriptors, descriptor))
        
    k = 128
    voc, variance = kmeans(descriptors, k, 1)
    print("Done doing K-means")
    
    im_features = np.zeros((len(des_list), k), "float32")
    for i in range(len(des_list)):
        print("Calculating distance for image "+str(i)+"...", end="\r")
        words, distance = vq(des_list[i][2],voc)
        for w in words:
            im_features[i][w] += 1
            
    nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0*len(des_list) + 1) / (1.0*nbr_occurences + 1)), 'float32')
    scaled_im_features = im_features * idf
            
    stdSlr = StandardScaler().fit(scaled_im_features)
    scaled_im_features = stdSlr.transform(scaled_im_features)
    
    return scaled_im_features

def linesvc(scaled_im_features, Y):
    clf = LinearSVC()
    clf.fit(X=scaled_im_features, y=Y)
    return clf

def svc_predict(clf, test):
    y_predict = clf.predict(test)
    return y_predict

def svc_save(clf):
    joblib.dump((clf),"svc.pkl")
    return

def LBP(des_list, X, Y):
    radius = 1
    n_points = 8
    a = 0
    print("Calculating all the descriptors and keypoints...")
    for i in X:
        a += 1
        print("Calculating for image "+str(a)+"/"+str(len(X)), end="\r")
        im  = cv2.imread(i)
        im = cv2.resize(im, (64, 64))
        gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY) 
        lbp = local_binary_pattern(gray, n_points, radius)
        des_list.append(lbp)       
    a2=0
    
    for j in des_list:
        if j is None:
            a2 = a2+1
            del des_list[a2]
            del X[a2]
            del Y[a2]
            a2 = a2-1
        else:
            a2 = a2+1
            print("Calculating for image "+str(a2)+"/"+str(len(des_list)), end="\r")
    return

def knn_lbp(des_list):
    a1 = 0
    print("Stacking all the descriptors in a numpy array...")
    descriptors = des_list[0]
    for descriptor in des_list[1:]:
        a1=a1+1
        print("Calculating for image ["+ str(a1)+"/"+str(len(des_list)) +"]", end="\r")
        descriptors = np.vstack((descriptors, descriptor))
        
    k = 128
    voc, variance = kmeans(descriptors, k, 1)
    print("Done doing K-means")
    
    im_features = np.zeros((len(des_list), k), "float32")
    for i in range(len(des_list)):
        print("Calculating distance for image "+str(i)+"...", end="\r")
        words, distance = vq(des_list[i],voc)
        for w in words:
            im_features[i][w] += 1
            
    nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0*len(des_list) + 1) / (1.0*nbr_occurences + 1)), 'float32')
    scaled_im_features = im_features * idf
            
    stdSlr = StandardScaler().fit(scaled_im_features)
    scaled_im_features = stdSlr.transform(scaled_im_features)
    
    return scaled_im_features

def svc_save_lbp(clf):
    joblib.dump((clf),"svc_lbp.pkl")
    return

In [3]:
X_train_sift = []
Y_train_sift = []
X_test_sift = []
Y_test_sift = []
X_val_sift = []
Y_val_sift = []
des_train_sift = []
des_test_sift = []        
des_val_sift = []
read('train.txt', X_train_sift, Y_train_sift)
read('test.txt', X_test_sift, Y_test_sift)
read('val.txt', X_val_sift, Y_val_sift)
sift(des_train_sift, X_train_sift, Y_train_sift)
sift(des_test_sift, X_test_sift, Y_test_sift)
sift(des_val_sift, X_val_sift, Y_val_sift)
scaled_train_sift = knn_sift(des_train_sift)
scaled_test_sift = knn_sift(des_test_sift)
scaled_val_sift = knn_sift(des_val_sift)
train_clf_sift=linesvc(scaled_train_sift, Y_train_sift)
svc_save(train_clf_sift)
test_predict_sift = svc_predict(train_clf_sift, scaled_test_sift)
val_predict_sift = svc_predict(train_clf_sift, scaled_val_sift)

Calculating all the descriptors and keypoints...
Calculating all the descriptors and keypoints...
Calculating all the descriptors and keypoints...
Stacking all the descriptors in a numpy array...
Done doing K-meansage [63316/63317]
Stacking all the descriptors in a numpy array...
Done doing K-meansage [448/449]
Stacking all the descriptors in a numpy array...
Done doing K-meansage [449/450]


C:\Users\cmpss\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [4]:
X_train_lbp = []
Y_train_lbp = []
X_test_lbp = []
Y_test_lbp = []
X_val_lbp = []
Y_val_lbp = []
des_train_lbp = []
des_test_lbp = []        
des_val_lbp = []
read('train.txt', X_train_lbp, Y_train_lbp)
read('test.txt', X_test_lbp, Y_test_lbp)
read('val.txt', X_val_lbp, Y_val_lbp)
LBP(des_train_lbp, X_train_lbp, Y_train_lbp)
LBP(des_test_lbp, X_test_lbp, Y_test_lbp)
LBP(des_val_lbp, X_val_lbp, Y_val_lbp)
scaled_train_lbp = knn_lbp(des_train_lbp)
scaled_test_lbp = knn_lbp(des_test_lbp)
scaled_val_lbp = knn_lbp(des_val_lbp)
train_clf_lbp=linesvc(scaled_train_lbp, Y_train_lbp)
svc_save_lbp(train_clf_lbp)
test_predict_lbp = svc_predict(train_clf_lbp, scaled_test_lbp)
val_predict_lbp = svc_predict(train_clf_lbp, scaled_val_lbp)

Calculating all the descriptors and keypoints...
Calculating all the descriptors and keypoints...
Calculating all the descriptors and keypoints...
Stacking all the descriptors in a numpy array...
Done doing K-meansage [63324/63325]
Stacking all the descriptors in a numpy array...
Done doing K-meansage [449/450]
Stacking all the descriptors in a numpy array...
Done doing K-meansage [449/450]


C:\Users\cmpss\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [50]:
print("sift精確度")
metrics.precision_score(val_predict_sift, Y_val_sift, average='micro')

svc準確度


0.028888888888888888

In [51]:
print("sift準確度")
accuracy_score(test_predict_sift, Y_test_sift)

sift準確度


0.017817371937639197

In [52]:
print("lbp精確度")
metrics.precision_score(val_predict_lbp, Y_val_lbp, average='micro')

lbp精確度


0.013333333333333334

In [53]:
print("lbp準確度")
accuracy_score(test_predict_lbp, Y_test_lbp)

lbp準確度


0.022222222222222223